In [8]:
import os

from google import genai
from google.genai import types
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [4]:
CONFIGS = {
    "model": "gemini-2.0-flash",  # "gemini-2.5-flash-preview-05-20",  # "gemini-2.5-pro-preview-05-06",
    "temperature": 0.1,
    "top_p": 0.95,
    "max_output_tokens": 250,
}

In [5]:
contents = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.

JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": [["cheese", "tomato sauce", "peperoni"]]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella.

JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": [["tomato sauce", "basil", "mozzarella"]]
}
```

Now, I would like a large pizza, with the first half cheese and mozzarella. \
And the other tomato sauce, ham and pineapple.

JSON Response:"""

In [3]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [7]:
response = client.models.generate_content(
    model=CONFIGS["model"],
    config=types.GenerateContentConfig(
        temperature=CONFIGS["temperature"],
        top_p=CONFIGS["top_p"],
        max_output_tokens=CONFIGS["max_output_tokens"],
    ),
    contents=contents,
)

print(response.text)

```json
{
  "size": "large",
  "type": "half_and_half",
  "ingredients": [
    ["cheese", "mozzarella"],
    ["tomato sauce", "ham", "pineapple"]
  ]
}
```


In [13]:
contents_2 = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.

JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": [["cheese", "tomato sauce", "peperoni"]]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella.

JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": [["tomato sauce", "basil", "mozzarella"]]
}
```

EXAMPLE:
I'd like a medium pizza, half with mushrooms and cheese, half with pepperoni and olives.

JSON Response:
```
{
"size": "medium",
"type": "half_half",
"ingredients": [["mushrooms", "cheese"], ["pepperoni", "olives"]]
}
```

Now, I would like a small pizza, with the first half cheese and mozzarella. \
And the other tomato sauce, ham and pineapple.

JSON Response:"""

In [14]:
response = client.models.generate_content(
    model=CONFIGS["model"],
    config=types.GenerateContentConfig(
        temperature=CONFIGS["temperature"],
        top_p=CONFIGS["top_p"],
        max_output_tokens=CONFIGS["max_output_tokens"],
    ),
    contents=contents_2,
)

print(response.text)

```json
{
"size": "small",
"type": "half_half",
"ingredients": [["cheese", "mozzarella"], ["tomato sauce", "ham", "pineapple"]]
}
```


In [19]:
contents_3 = """Please parse the customer's pizza order below and extract the relevant information.

Order: I would like a small pizza, with the first half cheese and mozzarella. \
And the other tomato sauce, ham and pineapple."""

In [20]:
from pydantic import BaseModel


class PizzaOrder(BaseModel):
    SIZE: str
    TYPE: str
    INGREDIENTS: list[list[str]]


response = client.models.generate_content(
    model=CONFIGS["model"],
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents=contents_3,
)

print(response.text)

{
  "SIZE": "small",
  "TYPE": "pizza",
  "INGREDIENTS": [
    ["cheese", "mozzarella"],
    ["tomato sauce", "ham", "pineapple"]
  ]
}


In [23]:
from enum import Enum


class PizzaSize(Enum):
    SMALL = "small"
    MEDIUM = "medium"
    LARGE = "large"


class PizzaType(Enum):
    NORMAL = "normal"
    HALF_HALF = "half_half"


class PizzaOrder(BaseModel):
    SIZE: PizzaSize
    TYPE: PizzaType
    INGREDIENTS: list[list[str]]


response = client.models.generate_content(
    model=CONFIGS["model"],
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents=contents_3,
)

print(response.text)

{
  "SIZE": "small",
  "TYPE": "half_half",
  "INGREDIENTS": [
    [
      "cheese",
      "mozzarella"
    ],
    [
      "tomato sauce",
      "ham",
      "pineapple"
    ]
  ]
}
